In [7]:
import tensorflow as tf
import numpy as np
import spectreader
from PIL import Image

In [17]:
NUM_THREADS = 4
k_nepochs=1
k_batchsize = 1
k_nbatches = 100
k_shownum = 3 # limits the number of images displayed

# data dependent parameters
k_nClass=2
height=256
width=856

#Reads data from the prepaired file of TFRecords
def getImage(fname, nepochs) :
    print ('global getImage')
    label, image = spectreader.getImage(fname, nepochs)

    #image=tf.reshape(tf.image.rgb_to_grayscale(image),[height*width]) #We are already in B&W
    image=tf.reshape(image,[height*width])

    # re-define label as a "one-hot" vector 
    # it will be [0,1] or [1,0] here. 
    # This approach can easily be extended to more classes.
    label=tf.stack(tf.one_hot(label-1, k_nClass))
    return label, image


In [18]:
# This runs without a session, but sets up several ops for the graph.
# This function does *not* get called during the session (as I understand it)!
target, data = getImage("data/train-00000-of-00001", k_nepochs)

global getImage
getImage data/train-00000-of-00001


In [19]:
imageBatch, labelBatch = tf.train.shuffle_batch(
    [data, target], batch_size=k_batchsize,
    num_threads=NUM_THREADS,
    
    enqueue_many=False, #IMPORTANT to get right, default=False
    capacity=10,  #1000,
    min_after_dequeue=5) #500


In [20]:

with tf.Session() as sess:
    #!!! need this line if there is a num_epochs other than none in getImage string_input_producer
    sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))
    
    # Create a coordinator, launch the queue runner threads.
    coord = tf.train.Coordinator()
    #XX  enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    enqueue_threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    shownum=0
    
    for step in xrange(k_nbatches): 
        if coord.should_stop():
            break
        #XX  data_batch, label_batch = sess.run(dequeue_op)
        data_batch, label_batch = sess.run([imageBatch, labelBatch])
        
        print('------  step: ' + str(step))
        print('data_batch tf.shape is ' + str(tf.shape(data_batch))) # 2, its a 2D array
        print('batch size is ' + str(data_batch.shape[0])) # (rows, colums)
        print('data size is ' + str(data_batch.shape[1])) #the length of a column
        print label_batch
        #print data_batch

        for i in range(k_batchsize) :
            if shownum < k_shownum :
                foo=Image.fromarray(np.reshape(data_batch[i]*255, (height, width)))
                foo.show()
                shownum += 1
        
    coord.request_stop()
    coord.join(enqueue_threads)

------  step: 0
data_batch tf.shape is Tensor("Shape_5:0", shape=(2,), dtype=int32)
batch size is 1
data size is 219136
[[ 1.  0.]]
------  step: 1
data_batch tf.shape is Tensor("Shape_6:0", shape=(2,), dtype=int32)
batch size is 1
data size is 219136
[[ 1.  0.]]
------  step: 2
data_batch tf.shape is Tensor("Shape_7:0", shape=(2,), dtype=int32)
batch size is 1
data size is 219136
[[ 0.  1.]]
------  step: 3
data_batch tf.shape is Tensor("Shape_8:0", shape=(2,), dtype=int32)
batch size is 1
data size is 219136
[[ 0.  1.]]
------  step: 4
data_batch tf.shape is Tensor("Shape_9:0", shape=(2,), dtype=int32)
batch size is 1
data size is 219136
[[ 1.  0.]]
------  step: 5
data_batch tf.shape is Tensor("Shape_10:0", shape=(2,), dtype=int32)
batch size is 1
data size is 219136
[[ 0.  1.]]
------  step: 6
data_batch tf.shape is Tensor("Shape_11:0", shape=(2,), dtype=int32)
batch size is 1
data size is 219136
[[ 0.  1.]]
------  step: 7
data_batch tf.shape is Tensor("Shape_12:0", shape=(2,), dt

OutOfRangeError: RandomShuffleQueue '_23_shuffle_batch_2/random_shuffle_queue' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: shuffle_batch_2 = QueueDequeueMany[_class=["loc:@shuffle_batch_2/random_shuffle_queue"], component_types=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_2/random_shuffle_queue, shuffle_batch_2/n)]]

Caused by op u'shuffle_batch_2', defined at:
  File "/Users/mclonce/anaconda3Mac/envs/tflow2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/mclonce/anaconda3Mac/envs/tflow2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/mclonce/anaconda3Mac/envs/tflow2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/mclonce/anaconda3Mac/envs/tflow2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in l